# Building sequential model file

In [14]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [15]:
import pickle

pickle_in = open("../Dataset/v4/X4.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("../Dataset/v4/y4.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

In [16]:
from tensorflow.keras.callbacks import TensorBoard
import time
import datetime
import os

In [17]:
NAME = "math-symbol-CNN"
log_dir='.\logs\\fit\math-symbol-CNN7\\'
 
tensorboard = TensorBoard(log_dir)
print(log_dir)

.\logs\fit\math-symbol-CNN7\


In [18]:
X.shape[1:]

(28, 28, 1)

In [19]:
num_classes=26
def model1():
    model = Sequential()

    model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    model.add(Dense(64))

    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X, y, batch_size=32, epochs=10, validation_split=0.2,callbacks=[tensorboard])
    
def model2():
    model = Sequential()
    model.add(Conv2D(32,(3,3), activation='relu',input_shape=X.shape[1:]))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(32,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(64,activation='tanh'))
    model.add(Dropout(0.2))
    
    model.add(Dense(num_classes,activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.fit(X, y, batch_size=32, epochs=15, validation_split=0.1,callbacks=[tensorboard])
    
model1()

Train on 62204 samples, validate on 15551 samples
Epoch 1/10
62204/62204 [==============================] - 39s 634us/sample - loss: 0.2417 - acc: 0.9294 - val_loss: 10.8540 - val_acc: 0.0759
Epoch 2/10
62204/62204 [==============================] - 39s 628us/sample - loss: 0.0844 - acc: 0.9757 - val_loss: 9.8471 - val_acc: 0.0749
Epoch 3/10
62204/62204 [==============================] - 39s 627us/sample - loss: 0.0584 - acc: 0.9835 - val_loss: 9.5005 - val_acc: 0.0744
Epoch 4/10
62204/62204 [==============================] - 39s 628us/sample - loss: 0.0458 - acc: 0.9870 - val_loss: 10.2021 - val_acc: 0.0743
Epoch 5/10
62204/62204 [==============================] - 39s 629us/sample - loss: 0.0388 - acc: 0.9887 - val_loss: 8.0279 - val_acc: 0.0687
Epoch 6/10
62204/62204 [==============================] - 39s 628us/sample - loss: 0.0329 - acc: 0.9902 - val_loss: 9.1545 - val_acc: 0.0763
Epoch 7/10
62204/62204 [==============================] - 39s 630us/sample - loss: 0.0310 - acc: 0.990

# tensorboard --logdir=ENTERLOGFOLDERPATH --host localhost --port 8088

In [24]:
import time
num_classes=26
results=[]
for conv2d_count in [1,2]:
    for dense_size in [128,256,512,1024,2048]:
        for dropout in [0.0,0.25,0.50,0.75]:
            model = Sequential()
            for i in range(conv2d_count):
                if i==0:
                    model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=X.shape[1:]))
                else:
                    model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))
            model.add(Flatten())
            model.add(Dense(dense_size,activation='tanh'))
            if dropout>0.0:
                model.add(Dropout(dropout))
            model.add(Dense(num_classes,activation='softmax'))
            
            model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            
            log_dir = '.\logsconv2d_%d-dense_%d-dropout_%.2f' % (conv2d_count,dense_size,dropout)
            tensorboard = TensorBoard(log_dir)
            
            start = time.time()
            
            model.fit(X,y,batch_size=32,epochs=10,verbose=0,validation_split=0.2,callbacks=[tensorboard])
#             score = model.evaluate(test_input,test_output,verbose=2)
            end=time.time()
            elapsed=end-start
            print('Conv2D count: %d, Dense size: %d, Dropout: %.2f , Time:%d sec' %  (conv2d_count, dense_size, dropout, elapsed ))

Conv2D count: 1, Dense size: 128, Dropout: 0.00 , Time:91 sec
Conv2D count: 1, Dense size: 128, Dropout: 0.25 , Time:95 sec


W0919 04:53:32.992662 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 128, Dropout: 0.50 , Time:95 sec
Conv2D count: 1, Dense size: 128, Dropout: 0.75 , Time:99 sec
Conv2D count: 1, Dense size: 256, Dropout: 0.00 , Time:104 sec
Conv2D count: 1, Dense size: 256, Dropout: 0.25 , Time:103 sec


W0919 05:00:27.006839 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 256, Dropout: 0.50 , Time:106 sec
Conv2D count: 1, Dense size: 256, Dropout: 0.75 , Time:104 sec
Conv2D count: 1, Dense size: 512, Dropout: 0.00 , Time:121 sec
Conv2D count: 1, Dense size: 512, Dropout: 0.25 , Time:124 sec


...

W0919 05:18:55.228289 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 1024, Dropout: 0.50 , Time:167 sec
Conv2D count: 1, Dense size: 1024, Dropout: 0.75 , Time:168 sec
Conv2D count: 1, Dense size: 2048, Dropout: 0.00 , Time:244 sec
Conv2D count: 1, Dense size: 2048, Dropout: 0.25 , Time:245 sec


W0919 05:34:01.135713 12456 nn_ops.py:4224] Large dropout rate: 0.75 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Conv2D count: 1, Dense size: 2048, Dropout: 0.50 , Time:246 sec


ResourceExhaustedError: OOM when allocating tensor with shape[5408,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node Adam_27/gradients/dense_54/MatMul_grad/MatMul_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
